In [ ]:
import pandas as pd
import sqlite3 as sql
import duckdb

In [6]:
conn = sql.connect('database.sqlite')
matches_df = pd.read_sql_query("SELECT * FROM Match", conn)

In [21]:
matches_df.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [ ]:
bookies_columns = []


for m in matches_df.columns:
    if m.endswith(('A', 'D', 'H')):
        bookies_columns.append(m)
        
# for bookies erase the last letter get only unique values
bookies_unique = set()
for col in bookies_columns:
    bookies_unique.add(col[:-1])
print('\n'.join(sorted(bookies_unique)))


B365
BS
BW
GB
IW
LB
PS
SJ
VC
WH


In [30]:
bookmakers = {
    "B365": {
        "name": "Bet365",
        "url": "https://www.bet365.com/",
        "country": "United Kingdom"
    },
    "BS": {
        "name": "Betsson",
        "url": "https://www.betsson.com/",
        "country": "Sweden"
    },
    "BW": {
        "name": "Bwin",
        "url": "https://www.bwin.com/",
        "country": "Austria"
    },
    "GB": {
        "name": "Gamebookers",
        "url": "https://www.gamebookers.com/",
        "country": "Antigua and Barbuda"
    },
    "IW": {
        "name": "Interwetten",
        "url": "https://www.interwetten.com/",
        "country": "Austria"
    },
    "LB": {
        "name": "Ladbrokes",
        "url": "https://www.ladbrokes.com/",
        "country": "United Kingdom"
    },
    "PS": {
        "name": "Pinnacle Sports",
        "url": "https://www.pinnacle.com/",
        "country": "Curaçao"
    },
    "SJ": {
        "name": "Stan James",
        "url": "https://www.stanjames.com/",
        "country": "United Kingdom"
    },
    "VC": {
        "name": "Victor Chandler (BetVictor)",
        "url": "https://www.betvictor.com/",
        "country": "Gibraltar"
    },
    "WH": {
        "name": "William Hill",
        "url": "https://www.williamhill.com/",
        "country": "United Kingdom"
    }
}


In [33]:
bookmakers_df = pd.DataFrame.from_dict(bookmakers, orient='index').reset_index()
bookmakers_df.rename(columns={'index': 'code'}, inplace=True)
bookmakers_df['id'] = range(1, len(bookmakers_df) + 1)
bookmakers_df.head()

,code,name,url,country,id
0,B365,Bet365,https://www.bet365.com/,United Kingdom,1
1,BS,Betsson,https://www.betsson.com/,Sweden,2
2,BW,Bwin,https://www.bwin.com/,Austria,3
3,GB,Gamebookers,https://www.gamebookers.com/,Antigua and Barbuda,4
4,IW,Interwetten,https://www.interwetten.com/,Austria,5


In [22]:
# from the dataframe get the match_id and the bookies columns
matches_bookies_df = matches_df[['match_api_id'] + bookies_columns]
matches_bookies_df.head()

,match_api_id,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,492473,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,492474,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,492475,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,492476,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,492477,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [24]:
matches_bookies_unpivoted = matches_bookies_df.melt(
    id_vars=['match_api_id'],
    value_vars=bookies_columns,
    var_name='bookie_odds_type',
    value_name='odds'
)
matches_bookies_unpivoted.sort_values(by=['match_api_id', 'bookie_odds_type'], inplace=True)
matches_bookies_unpivoted.head(20)

,match_api_id,bookie_odds_type,odds
56726,483129,B365A,3.75
30747,483129,B365D,3.10
4768,483129,B365H,2.10
758159,483129,BSA,3.75
732180,483129,BSD,2.88
706201,483129,BSH,2.00
134663,483129,BWA,3.55
108684,483129,BWD,3.00
82705,483129,BWH,2.05
680222,483129,GBA,3.75


In [26]:
matches_bookies_unpivoted['local'] = matches_bookies_unpivoted['bookie_odds_type'].str[-1]
matches_bookies_unpivoted['bookie'] = matches_bookies_unpivoted['bookie_odds_type'].str[:-1]
matches_bookies_unpivoted.head()

,match_api_id,bookie_odds_type,odds,local,bookie
56726,483129,B365A,3.75,A,B365
30747,483129,B365D,3.10,D,B365
4768,483129,B365H,2.10,H,B365
758159,483129,BSA,3.75,A,BS
732180,483129,BSD,2.88,D,BS


In [27]:
pivoted_df = matches_bookies_unpivoted.pivot_table(
    index=['match_api_id', 'bookie'],
    columns='local',
    values='odds'
).reset_index()
pivoted_df.head()

local,match_api_id,bookie,A,D,H
0,483129,B365,3.75,3.10,2.10
1,483129,BS,3.75,2.88,2.00
2,483129,BW,3.55,3.00,2.05
3,483129,GB,3.75,3.00,2.10
4,483129,IW,3.60,3.00,2.00


In [41]:
match_event_df = pd.DataFrame([{
    'id': 1,
    'name': 'Opening',
    'description': 'Odds at the match kickoff'
}])
match_event_df


pivoted_df['id_match_event'] = 1

In [40]:
def calculate_overround(home_odds, draw_odds, away_odds):
    """
    Calculate the overround given odds for home, draw, and away.
    Returns the overround rounded to 3 decimal places.
    """
    if home_odds and draw_odds and away_odds:
        overround = ((1/home_odds + 1/draw_odds + 1/away_odds) * 100) - 100
        return round(overround, 3)
    return None

pivoted_df['overround'] = pivoted_df.apply(
    lambda row: calculate_overround(row.get('H'), row.get('D'), row.get('A')),
    axis=1
)
pivoted_df.head()

local,match_api_id,bookie,A,D,H,match_event,overround
0,483129,B365,3.75,3.10,2.10,1,6.544
1,483129,BS,3.75,2.88,2.00,1,11.389
2,483129,BW,3.55,3.00,2.05,1,10.283
3,483129,GB,3.75,3.00,2.10,1,7.619
4,483129,IW,3.60,3.00,2.00,1,11.111


In [ ]:
pivoted_df = pivoted_df.merge(
    bookmakers_df[['code', 'id']],
    left_on='bookie',
    right_on='code',
    how='left'
).drop(columns=['code'])

pivoted_df.rename(columns={'id': 'id_bookmaker'}, inplace=True)
pivoted_df.head()

,match_api_id,bookie,A,D,H,match_event,overround,id_match_event,id_bookmaker
0,483129,B365,3.75,3.10,2.10,1,6.544,1,1
1,483129,BS,3.75,2.88,2.00,1,11.389,1,2
2,483129,BW,3.55,3.00,2.05,1,10.283,1,3
3,483129,GB,3.75,3.00,2.10,1,7.619,1,4
4,483129,IW,3.60,3.00,2.00,1,11.111,1,5


In [43]:
pivoted_df = pivoted_df.drop(columns=['bookie']).rename(columns={'match_api_id': 'id_match'})
pivoted_df.head()

,id_match,A,D,H,match_event,overround,id_match_event,id_bookmaker
0,483129,3.75,3.10,2.10,1,6.544,1,1
1,483129,3.75,2.88,2.00,1,11.389,1,2
2,483129,3.55,3.00,2.05,1,10.283,1,3
3,483129,3.75,3.00,2.10,1,7.619,1,4
4,483129,3.60,3.00,2.00,1,11.111,1,5


ModuleNotFoundError: No module named 'spark'